#### What is Databricks SQL Analytics?
* Databricks offers the convenience to build notebooks in Databricks Workspace using the SQL language, perform queries on data, build visualizations, build dashboards and connect to various BI tools. It offers up to 9X better price/performance for BI and reporting workloads.


* Advantages:
    - SQL-native user interface for data analytics 
    - Built in connectors for existing BI tools
    - Fine-grained performance
    - Fast query performance

#### Databricks workspace deployment on AWS
<img src='Databricks_Architecture.png' width=700>

* Create a cross-account role and an access policy in AWS.
    * In the AWS Console, go to IAM Services.
    * Under the Create role option, Choose the option Another AWS account .Enter the databricks account ID.
    <img src='Create_Role.png' width=700>
    * Under permissions Add the following inline JSON policy.
    <code>
    {
      "Version": "2012-10-17",
      "Statement": [
        {
          "Sid": "Stmt1403287045000",
          "Effect": "Allow",
          "Action": [
              "ec2:AssociateDhcpOptions",
              "ec2:AssociateIamInstanceProfile",
              "ec2:AssociateRouteTable",
              "ec2:AttachInternetGateway",
              "ec2:AttachVolume",
              "ec2:AuthorizeSecurityGroupEgress",
              "ec2:AuthorizeSecurityGroupIngress",
              "ec2:CancelSpotInstanceRequests",
              "ec2:CreateDhcpOptions",
              "ec2:CreateInternetGateway",
              "ec2:CreateKeyPair",
              "ec2:CreatePlacementGroup",
              "ec2:CreateRoute",
              "ec2:CreateSecurityGroup",
              "ec2:CreateSubnet",
              "ec2:CreateTags",
              "ec2:CreateVolume",
              "ec2:CreateVpc",
              "ec2:CreateVpcPeeringConnection",
              "ec2:DeleteInternetGateway",
              "ec2:DeleteKeyPair",
              "ec2:DeletePlacementGroup",
              "ec2:DeleteRoute",
              "ec2:DeleteRouteTable",
              "ec2:DeleteSecurityGroup",
              "ec2:DeleteSubnet",
              "ec2:DeleteTags",
              "ec2:DeleteVolume",
              "ec2:DeleteVpc",
              "ec2:DescribeAvailabilityZones",
              "ec2:DescribeIamInstanceProfileAssociations",
              "ec2:DescribeInstanceStatus",
              "ec2:DescribeInstances",
              "ec2:DescribePlacementGroups",
              "ec2:DescribePrefixLists",
              "ec2:DescribeReservedInstancesOfferings",
              "ec2:DescribeRouteTables",
              "ec2:DescribeSecurityGroups",
              "ec2:DescribeSpotInstanceRequests",
              "ec2:DescribeSpotPriceHistory",
              "ec2:DescribeSubnets",
              "ec2:DescribeVolumes",
              "ec2:DescribeVpcs",
              "ec2:DetachInternetGateway",
              "ec2:DisassociateIamInstanceProfile",
              "ec2:ModifyVpcAttribute",
              "ec2:ReplaceIamInstanceProfileAssociation",
              "ec2:RequestSpotInstances",
              "ec2:RevokeSecurityGroupEgress",
              "ec2:RevokeSecurityGroupIngress",
              "ec2:RunInstances",
              "ec2:TerminateInstances"
          ],
          "Resource": [
            "*"
          ]
        },
        {
          "Effect": "Allow",
          "Action": [
            "iam:CreateServiceLinkedRole",
            "iam:PutRolePolicy"
          ],
          "Resource": "arn:aws:iam::*:role/aws-service-role/spot.amazonaws.com/AWSServiceRoleForEC2Spot",
          "Condition": {
            "StringLike": {
              "iam:AWSServiceName": "spot.amazonaws.com"
            }
          }
        }
      ]
    }
    </code>
    
    - Review policy enter a policy name and click on Create Policy.
    - In the role summary, Copy the Role ARN.

* Go to your databricks account and under the AWS Account settings, paste your Role ARN.
<img src='Role_ARN.png' width=700>

#### Getting Started as a Databricks Workspace User / Deploying a Cluster
* Once you are in the databricks Workspace, Choose Clusters tab and Create Cluster option.
<img src='Deploy_Cluster.png' width=700>
* Under New Cluster option Databricks Runtime Version 7.3 which has support for Spark 3.0.1.
* Choose the Worker Node type. I have chosen m5a.xlarge which offers 16GB memory and 4 vCPUs.
<img src='Create_Cluster.png' width=700>
* Review parameters and click Create Cluster.

#### Create a new notebook
* Once your cluster is up and running, Choose the workspace tab, Under the option Workspace and under Create choose Notebook.
<img src='Create_Notebook.png' width=700>
* Give name to your notebook. Choose the default language as Python and select the newly created Cluster my-spark-cluster.

#### Create a Spark SQL table from CSV data
* Click the Data tab and click Add Data. 
* Under Data source section choose Upload file. This will upload your file under the DBFS Target directory /FileStore/tables/.
<img src='Create_Table.png' width=700>
* Run the following code to create a Spark SQL table from your CSV file.
<code>
%sql
DROP TABLE IF EXISTS covid_CA;
CREATE TABLE covid_CA USING CSV OPTIONS (path "/FileStore/tables/CA__covid19__latest.csv", header "true")
</code>
<img src='SQL_Table.png' width=700>

#### Querying the table using SparkSQL
* Printing the records.
<code>
%sql
SELECT * FROM covid_CA ORDER BY date DESC
</code>
<img src='SparkSQL_Query.png' width=700>
* Get the number of Confirmed Cases for each province.
<code>
%sql
SELECT prname AS Province, sum(numtoday) AS Number_of_Confirmed_Cases 
FROM covid_CA 
GROUP BY prname 
ORDER BY sum(numtoday) DESC
</code>
<img src='SparkSQL_Query2.png' width=700>

#### Querying the table by creating Spark dataframe
* Create a Spark Dataframe from CSV file.
<code>
covid_CA_df = spark.read.csv("/FileStore/tables/CA__covid19__latest.csv", header="true", inferSchema="true")
</code>
<img src='Dataframe_Query.png' width=700>
* Printing the records using PySpark dataframe.
<code>
covid_CA_df.display()
</code>
<img src='Dataframe_Query2.png' width=700>
* Get the Number of Confirmed Cases for Each Province using PySpark Dataframe.
<code>
from pyspark.sql.functions import sum as _sum

covid_CA_df \
  .groupBy('prname') \
  .agg(_sum('numtoday').alias('Total Confirmed Cases')) \
  .orderBy(_sum('numtoday').desc()) \
  .show()
</code>
<img src='Dataframe_Query3.png' width=700>

#### Visualize the data for insights
* Databricks supports various types of visualizations out of the box using the display and displayHTML functions.
<code>
display(
  covid_CA_df \
    .groupBy('prname') \
    .agg(_sum('numtoday').alias('Total Confirmed Cases')) \
    .orderBy(_sum('numtoday').desc())
)
</code>
<img src='Visualize.png' width=700>
* Click the arrow near the Bar Chart Icon.
<img src='Visualize2.png' width=700>
* Choose Bar option.
<img src='Visualize3.png' width=700>